In [1369]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [1370]:
from scipy.interpolate import interp1d
# resample target
def resample_cycle(cycle, num_points):
    """Resamples a gait cycle to have a fixed number of points using interpolation."""
    original_indices = np.linspace(0, 1, len(cycle))  # original indices based on the length of the cycle
    new_indices = np.linspace(0, 1, num_points)  # new indices to match the desired number of points
    interpolation_function = interp1d(original_indices, cycle, kind='linear')  # interpolate the data
    return interpolation_function(new_indices)  # return the resampled cycle

def rotate_cyclic(df, column_name, range):
    # Move the last value to the first position using pd.concat()
    return pd.concat([df[column_name].iloc[-range:], df[column_name].iloc[:-range]]).reset_index(drop=True)


In [1371]:
leg_0_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_me_0.csv')
leg_1_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_me_1.csv')
leg_2_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_me_2.csv')

# leg_0_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_si_0.csv')
# leg_1_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_si_1.csv')
# leg_2_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_si_2.csv')

# leg_0_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_ma21_0.csv')
# leg_1_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_ma21_1.csv')
# leg_2_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_ma21_2.csv')

print(leg_0_df)

        tr     cr     fr      z
0    0.827  0.295 -1.367  0.000
1    0.825  0.297 -1.371  0.000
2    0.823  0.298 -1.375  0.000
3    0.821  0.299 -1.379  0.000
4    0.819  0.301 -1.384  0.000
..     ...    ...    ...    ...
719  0.841  0.287 -1.337  0.001
720  0.839  0.287 -1.340  0.001
721  0.838  0.288 -1.344  0.001
722  0.836  0.289 -1.347  0.001
723  0.834  0.290 -1.351  0.001

[724 rows x 4 columns]


In [1372]:

leg_0_df.rename(columns={'tr': 'TR0', 'cr': 'CR0', 'fr': 'FR0', 'z': 'ZR0'}, inplace=True)
leg_1_df.rename(columns={'tr': 'TR1', 'cr': 'CR1', 'fr': 'FR1', 'z': 'ZR1'}, inplace=True)
leg_2_df.rename(columns={'tr': 'TR2', 'cr': 'CR2', 'fr': 'FR2', 'z': 'ZR2'}, inplace=True)

leg_0_df[['TL0', 'CL0', 'FL0', 'ZL0']] = leg_0_df[['TR0', 'CR0', 'FR0', 'ZR0']]
leg_1_df[['TL1', 'CL1', 'FL1', 'ZL1']] = leg_1_df[['TR1', 'CR1', 'FR1', 'ZR1']]
leg_2_df[['TL2', 'CL2', 'FL2', 'ZL2']] = leg_2_df[['TR2', 'CR2', 'FR2', 'ZR2']]


print(leg_0_df)

       TR0    CR0    FR0    ZR0    TL0    CL0    FL0    ZL0
0    0.827  0.295 -1.367  0.000  0.827  0.295 -1.367  0.000
1    0.825  0.297 -1.371  0.000  0.825  0.297 -1.371  0.000
2    0.823  0.298 -1.375  0.000  0.823  0.298 -1.375  0.000
3    0.821  0.299 -1.379  0.000  0.821  0.299 -1.379  0.000
4    0.819  0.301 -1.384  0.000  0.819  0.301 -1.384  0.000
..     ...    ...    ...    ...    ...    ...    ...    ...
719  0.841  0.287 -1.337  0.001  0.841  0.287 -1.337  0.001
720  0.839  0.287 -1.340  0.001  0.839  0.287 -1.340  0.001
721  0.838  0.288 -1.344  0.001  0.838  0.288 -1.344  0.001
722  0.836  0.289 -1.347  0.001  0.836  0.289 -1.347  0.001
723  0.834  0.290 -1.351  0.001  0.834  0.290 -1.351  0.001

[724 rows x 8 columns]


In [1373]:
# phase_shift_1 = 15  
# phase_shift_2 = 300  
# phase_shift_3 = 80

phase_shift_1 = 50  
phase_shift_2 = 50  
phase_shift_3 = 0

leg_0_df['TR0'] = rotate_cyclic(leg_0_df, 'TR0', phase_shift_1)
leg_0_df['CR0'] = rotate_cyclic(leg_0_df, 'CR0', phase_shift_1)
leg_0_df['FR0'] = rotate_cyclic(leg_0_df, 'FR0', phase_shift_1)
leg_0_df['ZR0'] = rotate_cyclic(leg_0_df, 'ZR0', phase_shift_1)

leg_1_df['TR1'] = rotate_cyclic(leg_1_df, 'TR1', phase_shift_2 + phase_shift_3)
leg_1_df['CR1'] = rotate_cyclic(leg_1_df, 'CR1', phase_shift_2 + phase_shift_3)
leg_1_df['FR1'] = rotate_cyclic(leg_1_df, 'FR1', phase_shift_2 + phase_shift_3)
leg_1_df['ZR1'] = rotate_cyclic(leg_1_df, 'ZR1', phase_shift_2 + phase_shift_3)

leg_2_df['TR2'] = rotate_cyclic(leg_2_df, 'TR2', phase_shift_1)
leg_2_df['CR2'] = rotate_cyclic(leg_2_df, 'CR2', phase_shift_1)
leg_2_df['FR2'] = rotate_cyclic(leg_2_df, 'FR2', phase_shift_1)
leg_2_df['ZR2'] = rotate_cyclic(leg_2_df, 'ZR2', phase_shift_1)


leg_0_df['TL0'] = rotate_cyclic(leg_0_df, 'TL0', phase_shift_2)
leg_0_df['CL0'] = rotate_cyclic(leg_0_df, 'CL0', phase_shift_2)
leg_0_df['FL0'] = rotate_cyclic(leg_0_df, 'FL0', phase_shift_2)
leg_0_df['ZL0'] = rotate_cyclic(leg_0_df, 'ZL0', phase_shift_2)

leg_1_df['TL1'] = rotate_cyclic(leg_1_df, 'TL1', phase_shift_1)
leg_1_df['CL1'] = rotate_cyclic(leg_1_df, 'CL1', phase_shift_1)
leg_1_df['FL1'] = rotate_cyclic(leg_1_df, 'FL1', phase_shift_1)
leg_1_df['ZL1'] = rotate_cyclic(leg_1_df, 'ZL1', phase_shift_1)

leg_2_df['TL2'] = rotate_cyclic(leg_2_df, 'TL2', phase_shift_2 + phase_shift_3)
leg_2_df['CL2'] = rotate_cyclic(leg_2_df, 'CL2', phase_shift_2 + phase_shift_3)
leg_2_df['FL2'] = rotate_cyclic(leg_2_df, 'FL2', phase_shift_2 + phase_shift_3)
leg_2_df['ZL2'] = rotate_cyclic(leg_2_df, 'ZL2', phase_shift_2 + phase_shift_3)

In [1374]:
# Apply the condition
leg_0_df['foot_R0'] = leg_0_df['ZR0'].apply(lambda x: 0.0 if x > 0.0 else 1.0)
leg_1_df['foot_R1'] = leg_1_df['ZR1'].apply(lambda x: 0.0 if x > 0.0 else 1.0)
leg_2_df['foot_R2'] = leg_2_df['ZR2'].apply(lambda x: 0.0 if x > 0.0 else 1.0)

leg_0_df['foot_L0'] = leg_0_df['ZL0'].apply(lambda x: 0.0 if x > 0.0 else 1.0)
leg_1_df['foot_L1'] = leg_1_df['ZL1'].apply(lambda x: 0.0 if x > 0.0 else 1.0)
leg_2_df['foot_L2'] = leg_2_df['ZL2'].apply(lambda x: 0.0 if x > 0.0 else 1.0)


# # Apply the moving average for each dataframe
# window_size = 50
# leg_0_df['foot_R0'] = leg_0_df['foot_R0'].rolling(window=window_size, min_periods=1).mean()
# leg_1_df['foot_R1'] = leg_1_df['foot_R1'].rolling(window=window_size, min_periods=1).mean()
# leg_2_df['foot_R2'] = leg_2_df['foot_R2'].rolling(window=window_size, min_periods=1).mean()

# leg_0_df['foot_L0'] = leg_0_df['foot_L0'].rolling(window=window_size, min_periods=1).mean()
# leg_1_df['foot_L1'] = leg_1_df['foot_L1'].rolling(window=window_size, min_periods=1).mean()
# leg_2_df['foot_L2'] = leg_2_df['foot_L2'].rolling(window=window_size, min_periods=1).mean()


- Modify Swimming Pattern

In [1375]:
def apply_moving_average(df, columns, window):
    for col in columns:
        # center=True prevents the signal from shifting to the right (lag)
        # min_periods=1 ensures we don't get NaNs at the start/end
        df[col] = df[col].rolling(window=window, center=True, min_periods=1).mean()
    return df

In [1376]:
leg_0_st_indx = leg_0_df.index[leg_0_df['foot_R0'] > 0.5]
leg_0_sw_indx = leg_0_df.index[leg_0_df['foot_R0'] < 0.5]
leg_1_st_indx = leg_1_df.index[leg_1_df['foot_R1'] > 0.5]
leg_1_sw_indx = leg_1_df.index[leg_1_df['foot_R1'] < 0.5]
leg_2_st_indx = leg_2_df.index[leg_2_df['foot_R2'] > 0.5]
leg_2_sw_indx = leg_2_df.index[leg_2_df['foot_R2'] < 0.5]

In [1377]:
T_scale_factor = 2.5
leg_0_df['TR0'] = ((leg_0_df['TR0'] - 0.5) * T_scale_factor) + 0.5
leg_1_df['TR1'] = leg_1_df['TR1'] * T_scale_factor
leg_2_df['TR2'] = ((leg_2_df['TR2'] + 0.5) * (T_scale_factor - 0.5)) - 0.5


C_scale_factor_st = 1.5
C_scale_factor_sw = 2
leg_0_df.loc[leg_0_st_indx, 'CR0'] = (leg_0_df.loc[leg_0_st_indx, 'CR0'] - 0.7) * C_scale_factor_st + 0.7
leg_0_df.loc[leg_0_sw_indx, 'CR0'] = (leg_0_df.loc[leg_0_sw_indx, 'CR0'] - 0.7) * C_scale_factor_sw + 0.7

C_scale_factor_st = 1.5
C_scale_factor_sw = 1.8
leg_1_df.loc[leg_1_sw_indx, 'CR1'] = (leg_1_df.loc[leg_1_sw_indx, 'CR1']) * C_scale_factor_sw 

C_scale_factor_st = 0
C_scale_factor_sw = 2
leg_2_df.loc[leg_2_st_indx, 'CR2'] = (leg_2_df.loc[leg_2_st_indx, 'CR2']) * C_scale_factor_st 
leg_2_df.loc[leg_2_sw_indx, 'CR2'] = (leg_2_df.loc[leg_2_sw_indx, 'CR2'] ) * C_scale_factor_sw 



F_scale_factor_st = 0
F_scale_factor_sw = 1.2
leg_0_df.loc[leg_0_st_indx, 'FR0'] = (leg_0_df.loc[leg_0_st_indx, 'FR0'] ) * F_scale_factor_st 
leg_0_df.loc[leg_0_sw_indx, 'FR0'] = (leg_0_df.loc[leg_0_sw_indx, 'FR0'] ) * F_scale_factor_sw 

F_scale_factor_st = 0
F_scale_factor_sw = 1.5
leg_1_df.loc[leg_1_st_indx, 'FR1'] = (leg_1_df.loc[leg_1_st_indx, 'FR1'] ) * F_scale_factor_st 
leg_1_df.loc[leg_1_sw_indx, 'FR1'] = (leg_1_df.loc[leg_1_sw_indx, 'FR1'] ) * F_scale_factor_sw 

F_scale_factor_st = 0
F_scale_factor_sw = 1.5
leg_2_df.loc[leg_2_st_indx, 'FR2'] = (leg_2_df.loc[leg_2_st_indx, 'FR2'] ) * F_scale_factor_st 
leg_2_df.loc[leg_2_sw_indx, 'FR2'] = (leg_2_df.loc[leg_2_sw_indx, 'FR2'] ) * F_scale_factor_sw



In [1378]:
leg_0_df['TL0'] = ((leg_0_df['TL0'] - 0.5) * T_scale_factor) + 0.5
leg_1_df['TL1'] = leg_1_df['TL1'] * T_scale_factor
leg_2_df['TL2'] = ((leg_2_df['TL2'] + 0.5) * (T_scale_factor - 0.5)) - 0.5



C_scale_factor_st = 1.5
C_scale_factor_sw = 2
leg_0_df.loc[leg_0_st_indx, 'CL0'] = (leg_0_df.loc[leg_0_st_indx, 'CL0'] - 0.7) * C_scale_factor_st + 0.7
leg_0_df.loc[leg_0_sw_indx, 'CL0'] = (leg_0_df.loc[leg_0_sw_indx, 'CL0'] - 0.7) * C_scale_factor_sw + 0.7

C_scale_factor_st = 1.5
C_scale_factor_sw = 1.8
leg_1_df.loc[leg_1_sw_indx, 'CL1'] = (leg_1_df.loc[leg_1_sw_indx, 'CL1']) * C_scale_factor_sw 

C_scale_factor_st = 0
C_scale_factor_sw = 2
leg_2_df.loc[leg_2_st_indx, 'CL2'] = (leg_2_df.loc[leg_2_st_indx, 'CL2']) * C_scale_factor_st
leg_2_df.loc[leg_2_sw_indx, 'CL2'] = (leg_2_df.loc[leg_2_sw_indx, 'CL2'] ) * C_scale_factor_sw 



F_scale_factor_st = 0
F_scale_factor_sw = 1.2
leg_0_df.loc[leg_0_st_indx, 'FL0'] = (leg_0_df.loc[leg_0_st_indx, 'FL0'] ) * F_scale_factor_st 
leg_0_df.loc[leg_0_sw_indx, 'FL0'] = (leg_0_df.loc[leg_0_sw_indx, 'FL0'] ) * F_scale_factor_sw 

F_scale_factor_st = 0
F_scale_factor_sw = 1.5
leg_1_df.loc[leg_1_st_indx, 'FL1'] = (leg_1_df.loc[leg_1_st_indx, 'FL1'] ) * F_scale_factor_st 
leg_1_df.loc[leg_1_sw_indx, 'FL1'] = (leg_1_df.loc[leg_1_sw_indx, 'FL1'] ) * F_scale_factor_sw 

F_scale_factor_st = 0
F_scale_factor_sw = 1.5
leg_2_df.loc[leg_2_st_indx, 'FL2'] = (leg_2_df.loc[leg_2_st_indx, 'FL2'] ) * F_scale_factor_st 
leg_2_df.loc[leg_2_sw_indx, 'FL2'] = (leg_2_df.loc[leg_2_sw_indx, 'FL2'] ) * F_scale_factor_sw

In [1379]:
window_size = 100
joints_leg_0 = ['TR0', 'CR0', 'FR0']
joints_leg_1 = ['TR1', 'CR1', 'FR1']
joints_leg_2 = ['TR2', 'CR2', 'FR2']
leg_0_df = apply_moving_average(leg_0_df, joints_leg_0, window_size)
leg_1_df = apply_moving_average(leg_1_df, joints_leg_1, window_size)
leg_2_df = apply_moving_average(leg_2_df, joints_leg_2, window_size)

joints_leg_0 = ['TL0', 'CL0', 'FL0']
joints_leg_1 = ['TL1', 'CL1', 'FL1']
joints_leg_2 = ['TL2', 'CL2', 'FL2']
leg_0_df = apply_moving_average(leg_0_df, joints_leg_0, window_size)
leg_1_df = apply_moving_average(leg_1_df, joints_leg_1, window_size)
leg_2_df = apply_moving_average(leg_2_df, joints_leg_2, window_size)

In [1380]:

fig = go.Figure()
fig = make_subplots(rows=4, cols=3, shared_xaxes=False)
fig.update_layout(height=1000, width=1100, title="Path plot")
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))  # Reduce margins

fig.add_trace(go.Scatter( y=leg_0_df['TR0'], mode='lines', name=f'TR0'), row=1, col=1)
fig.add_trace(go.Scatter( y=leg_0_df['TL0'], mode='lines', name=f'TL0'), row=1, col=1)

fig.add_trace(go.Scatter( y=leg_0_df['CR0'], mode='lines', name=f'CR0'), row=2, col=1)
fig.add_trace(go.Scatter( y=leg_0_df['CL0'], mode='lines', name=f'CL0'), row=2, col=1)

fig.add_trace(go.Scatter( y=leg_0_df['FR0'], mode='lines', name=f'FR0'), row=3, col=1)
fig.add_trace(go.Scatter( y=leg_0_df['FL0'], mode='lines', name=f'FL0'), row=3, col=1)

fig.add_trace(go.Scatter( y=leg_0_df['ZR0'], mode='lines', name=f'ZR0'), row=4, col=1)
fig.add_trace(go.Scatter( y=leg_0_df['ZL0'], mode='lines', name=f'ZL0'), row=4, col=1)

fig.add_trace(go.Scatter( y=leg_0_df['foot_R0'], mode='lines', name=f'foot_R0'), row=4, col=1)
fig.add_trace(go.Scatter( y=leg_0_df['foot_L0'], mode='lines', name=f'foot_L0'), row=4, col=1)



fig.add_trace(go.Scatter( y=leg_1_df['TR1'], mode='lines', name=f'TR1'), row=1, col=2)
fig.add_trace(go.Scatter( y=leg_1_df['TL1'], mode='lines', name=f'TL1'), row=1, col=2)

fig.add_trace(go.Scatter( y=leg_1_df['CR1'], mode='lines', name=f'CR1'), row=2, col=2)
fig.add_trace(go.Scatter( y=leg_1_df['CL1'], mode='lines', name=f'CL1'), row=2, col=2)

fig.add_trace(go.Scatter( y=leg_1_df['FR1'], mode='lines', name=f'FR1'), row=3, col=2)
fig.add_trace(go.Scatter( y=leg_1_df['FL1'], mode='lines', name=f'FL1'), row=3, col=2)

fig.add_trace(go.Scatter( y=leg_1_df['ZR1'], mode='lines', name=f'ZR1'), row=4, col=2)
fig.add_trace(go.Scatter( y=leg_1_df['ZL1'], mode='lines', name=f'ZL1'), row=4, col=2)

fig.add_trace(go.Scatter( y=leg_1_df['foot_R1'], mode='lines', name=f'foot_R1'), row=4, col=2)
fig.add_trace(go.Scatter( y=leg_1_df['foot_L1'], mode='lines', name=f'foot_L1'), row=4, col=2)



fig.add_trace(go.Scatter( y=leg_2_df['TR2'], mode='lines', name=f'TR2'), row=1, col=3)
fig.add_trace(go.Scatter( y=leg_2_df['TL2'], mode='lines', name=f'TL2'), row=1, col=3)

fig.add_trace(go.Scatter( y=leg_2_df['CR2'], mode='lines', name=f'CR2'), row=2, col=3)
fig.add_trace(go.Scatter( y=leg_2_df['CL2'], mode='lines', name=f'CL2'), row=2, col=3)

fig.add_trace(go.Scatter( y=leg_2_df['FR2'], mode='lines', name=f'FR2'), row=3, col=3)
fig.add_trace(go.Scatter( y=leg_2_df['FL2'], mode='lines', name=f'FL2'), row=3, col=3)

fig.add_trace(go.Scatter( y=leg_2_df['ZR2'], mode='lines', name=f'ZR2'), row=4, col=3)
fig.add_trace(go.Scatter( y=leg_2_df['ZL2'], mode='lines', name=f'ZL2'), row=4, col=3)

fig.add_trace(go.Scatter( y=leg_2_df['foot_R2'], mode='lines', name=f'foot_R2'), row=4, col=3)
fig.add_trace(go.Scatter( y=leg_2_df['foot_L2'], mode='lines', name=f'foot_L2'), row=4, col=3)

In [1381]:
print(leg_0_df.keys())

Index(['TR0', 'CR0', 'FR0', 'ZR0', 'TL0', 'CL0', 'FL0', 'ZL0', 'foot_R0',
       'foot_L0'],
      dtype='object')


In [1382]:
from include.cpg_rbf.cpg_so2 import CPG_SO2
# from include.cpg_rbf.cpg_afdc import CPG_AFDC
from include.cpg_rbf.rbf import RBF

rbf = RBF()
cpg = CPG_SO2()
cpg_one_cycle = cpg.generate_cpg_one_cycle(0.05)    # lowest phi value
out0_cpg_one_cycle = cpg_one_cycle['out0_cpg_one_cycle'][:] # Use full cycle of CPG
out1_cpg_one_cycle = cpg_one_cycle['out1_cpg_one_cycle'][:] # Use full cycle of CPG
cpg_cycle_length = len(out0_cpg_one_cycle)
cpg_kernel = rbf.construct_kernels_with_cpg_one_cycle(out0_cpg_one_cycle, out1_cpg_one_cycle, cpg_cycle_length) # construct kernels with cpg one cycle

In [1383]:
# Define dataframes and corresponding column groups
dataframes = [leg_0_df, leg_1_df, leg_2_df, leg_0_df, leg_1_df, leg_2_df]
column_groups = [
    ['TR0', 'CR0', 'FR0', 'ZR0'],
    ['TR1', 'CR1', 'FR1', 'ZR1'],
    ['TR2', 'CR2', 'FR2', 'ZR2'],
    ['TL0', 'CL0', 'FL0', 'ZL0'],
    ['TL1', 'CL1', 'FL1', 'ZL1'],
    ['TL2', 'CL2', 'FL2', 'ZL2']
]

imitated_paths = {}
imitated_weights = {}

# Loop through each dataframe and corresponding column group
for df, columns in zip(dataframes, column_groups):
    for col in columns:
        # Resample cycle for the current column
        target = resample_cycle(np.array(df[col]), cpg_cycle_length)
        
        # Imitate path and get weights using RBF
        imitated_paths[col] = rbf.imitate_path_by_learning(target)
        imitated_weights[col] = rbf.get_rbf_weight()
        
print(imitated_weights.keys())
np.savez('imitated_stick_insect_swim_weights_me.npz', **imitated_weights)

last iteration: 13
last iteration: 91
last iteration: 106
last iteration: 135
last iteration: 223
last iteration: 63
last iteration: 122
last iteration: 493
last iteration: 146
last iteration: 229
dict_keys(['TR0', 'CR0', 'FR0', 'ZR0', 'TR1', 'CR1', 'FR1', 'ZR1', 'TR2', 'CR2', 'FR2', 'ZR2', 'TL0', 'CL0', 'FL0', 'ZL0', 'TL1', 'CL1', 'FL1', 'ZL1', 'TL2', 'CL2', 'FL2', 'ZL2'])


In [1384]:
rbf_f = RBF()
out0_cpg_one_cycle_f = cpg_one_cycle['out0_cpg_one_cycle'][:] # Use full cycle of CPG
out1_cpg_one_cycle_f = cpg_one_cycle['out1_cpg_one_cycle'][:] # Use full cycle of CPG
cpg_cycle_length_f = len(out0_cpg_one_cycle_f)
cpg_kernel_f = rbf_f.construct_kernels_with_cpg_one_cycle(out0_cpg_one_cycle_f, out1_cpg_one_cycle_f, cpg_cycle_length_f) # construct kernels with cpg one cycle

In [1385]:
imitated_force = {}
imitated_force_weights = {}
dataframes = [leg_0_df, leg_1_df, leg_2_df, leg_0_df, leg_1_df, leg_2_df]
column_groups = [
    ['foot_R0'],
    ['foot_R1'],
    ['foot_R2'],
    ['foot_L0'],
    ['foot_L1'],
    ['foot_L2']
]
for df, columns in zip(dataframes, column_groups):
    for col in columns:
        # Resample cycle for the current column
        target = resample_cycle(np.array(df[col][:]), cpg_cycle_length_f)
        
        # Imitate path and get weights using RBF
        imitated_force[col] = rbf_f.imitate_path_by_learning(target)
        imitated_force_weights[col] = rbf_f.get_rbf_weight()
        
print(imitated_force_weights.keys())
np.savez('imitated_stick_insect_swim_weights_me_force.npz', **imitated_force_weights)

dict_keys(['foot_R0', 'foot_R1', 'foot_R2', 'foot_L0', 'foot_L1', 'foot_L2'])


In [1386]:
# data = np.load('imitated_stick_insect_right_leg_weights_me.npz')
# print(data.files)

In [1387]:

# fig = go.Figure()
# fig = make_subplots(rows=6, cols=1, shared_xaxes=False)
# fig.update_layout(height=800, width=1100, title="CPG plot")
# fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))  # Reduce margins

# # for i in range(len(cpg_kernel)):
# #     fig.add_trace(go.Scatter( y=cpg_kernel[i], mode='lines', name=f'kernels{i}'), row=2, col=1)

# fig.add_trace(go.Scatter( y=out0_cpg_one_cycle, mode='lines', name=f'out0_cpg_one_cycle'), row=1, col=1)
# fig.add_trace(go.Scatter( y=out1_cpg_one_cycle, mode='lines', name=f'out1_cpg_one_cycle'), row=1, col=1)

# fig.add_trace(go.Scatter( y=imitated_paths['TR0'], mode='lines', name=f'TR0 (CPG-RBF)'), row=3, col=1)
# fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_0_df['TR0']), cpg_cycle_length), mode='lines', name=f'TR0'), row=3, col=1)

# fig.add_trace(go.Scatter( y=imitated_paths['CR0'], mode='lines', name=f'CR0 (CPG-RBF)'), row=4, col=1)
# fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_0_df['CR0']), cpg_cycle_length), mode='lines', name=f'CR0'), row=4, col=1)

# fig.add_trace(go.Scatter( y=imitated_paths['FR0'], mode='lines', name=f'FR0 (CPG-RBF)'), row=5, col=1)
# fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_0_df['FR0']), cpg_cycle_length), mode='lines', name=f'FR0'), row=5, col=1)

# fig.add_trace(go.Scatter( y=imitated_force['F0'], mode='lines', name=f'F0 (CPG-RBF)'), row=6, col=1)
# fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_0_df['F0']), cpg_cycle_length), mode='lines', name=f'F0'), row=6, col=1)

# # fig.add_trace(go.Scatter( y=imitated_paths['TR1'], mode='lines', name=f'TR1 (CPG-RBF)'), row=3, col=1)
# # fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_1_df['TR1']), cpg_cycle_length), mode='lines', name=f'TR1'), row=3, col=1)

# # fig.add_trace(go.Scatter( y=imitated_paths['CR1'], mode='lines', name=f'CR1 (CPG-RBF)'), row=4, col=1)
# # fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_1_df['CR1']), cpg_cycle_length), mode='lines', name=f'CR1'), row=4, col=1)

# # fig.add_trace(go.Scatter( y=imitated_paths['FR1'], mode='lines', name=f'FR1 (CPG-RBF)'), row=5, col=1)
# # fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_1_df['FR1']), cpg_cycle_length), mode='lines', name=f'FR1'), row=5, col=1)

In [1388]:
# imitated_weights = np.load('imitated_stick_insect_right_leg_weights_me.npz')
# print(imitated_weights.files)

# # Access each array by its key
# for key in imitated_weights.files:
#     print(f"{key}: {imitated_weights[key]}")

In [1389]:
# cpg = CPG_SO2()
# cpg_output_0 = []
# cpg_output_1 = []
# online_imitated_paths = {'TR0':[], 'CR0':[], 'FR0':[], 'Z0':[], 'F0':[]}
# for i in range(1000):
#     cpg.s = 1.0
#     cpg.update_cpg(0.05)      # phi would be equal when learning

#     cpg_output_0.append(cpg.out0_t)
#     cpg_output_1.append(cpg.out1_t)
    
#     online_imitated_paths['TR0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['TR0']))
#     online_imitated_paths['CR0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['CR0']))
#     online_imitated_paths['FR0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['FR0']))
#     online_imitated_paths['F0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['F0']))

# fig = go.Figure()
# fig = make_subplots(rows=5, cols=1, shared_xaxes=True)
# fig.update_layout(height=600, width=1100, title="CPG plot")
# fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))  # Reduce margins

# fig.add_trace(go.Scatter( y=cpg_output_0, mode='lines', name=f'cpg_output_0'), row=1, col=1)
# fig.add_trace(go.Scatter( y=cpg_output_1, mode='lines', name=f'cpg_output_1'), row=1, col=1)
# fig.add_trace(go.Scatter( y=online_imitated_paths['TR0'], mode='lines', name=f'TR0 (CPG-RBF)'), row=2, col=1)
# fig.add_trace(go.Scatter( y=online_imitated_paths['CR0'], mode='lines', name=f'CR0 (CPG-RBF)'), row=3, col=1)
# fig.add_trace(go.Scatter( y=online_imitated_paths['FR0'], mode='lines', name=f'FR0 (CPG-RBF)'), row=4, col=1)
# fig.add_trace(go.Scatter( y=online_imitated_paths['F0'], mode='lines', name=f'F0 (CPG-RBF)'), row=5, col=1)




In [1390]:
# from include.cpg_rbf.cpg_so2 import CPG_SO2
# # from include.cpg_rbf.cpg_afdc import CPG_AFDC
# from include.cpg_rbf.rbf import RBF

# rbf = RBF()
# cpg = CPG_SO2()
# cpg_one_cycle = cpg.generate_cpg_one_cycle(0.05)    # lowest phi value
# out0_cpg_one_cycle = cpg_one_cycle['out0_cpg_one_cycle'][:] # Use full cycle of CPG
# out1_cpg_one_cycle = cpg_one_cycle['out1_cpg_one_cycle'][:] # Use full cycle of CPG
# cpg_cycle_length = len(out0_cpg_one_cycle)
# cpg_kernel = rbf.construct_kernels_with_cpg_one_cycle(out0_cpg_one_cycle, out1_cpg_one_cycle, cpg_cycle_length) # construct kernels with cpg one cycle

In [1391]:
# cpg = CPG_SO2()
# cpg_output_0 = []
# cpg_output_1 = []
# online_imitated_paths = {'TR1':[], 'CR1':[], 'FR1':[], 'Z1':[], 'F1':[]}
# for i in range(1000):
#     cpg.s = 1.0
#     cpg.update_cpg(0.05)      # phi would be equal when learning

#     cpg_output_0.append(cpg.out0_t)
#     cpg_output_1.append(cpg.out1_t)
    
#     online_imitated_paths['TR1'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['TR1']))
#     online_imitated_paths['CR1'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['CR1']))
#     online_imitated_paths['FR1'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['FR1']))
#     online_imitated_paths['F1'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_force_weights['F1']))

# fig = go.Figure()
# fig = make_subplots(rows=5, cols=1, shared_xaxes=True)
# fig.update_layout(height=600, width=1100, title="CPG plot")
# fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))  # Reduce margins

# fig.add_trace(go.Scatter( y=cpg_output_0, mode='lines', name=f'cpg_output_0'), row=1, col=1)
# fig.add_trace(go.Scatter( y=cpg_output_1, mode='lines', name=f'cpg_output_1'), row=1, col=1)
# fig.add_trace(go.Scatter( y=online_imitated_paths['TR1'], mode='lines', name=f'TR0 (CPG-RBF)'), row=2, col=1)
# fig.add_trace(go.Scatter( y=online_imitated_paths['CR1'], mode='lines', name=f'CR0 (CPG-RBF)'), row=3, col=1)
# fig.add_trace(go.Scatter( y=online_imitated_paths['FR1'], mode='lines', name=f'FR0 (CPG-RBF)'), row=4, col=1)
# fig.add_trace(go.Scatter( y=online_imitated_paths['F1'], mode='lines', name=f'F0 (CPG-RBF)'), row=5, col=1)
